In [7]:
%pip install -q --upgrade torch

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.6 requires torchvision>=0.11, which is not installed.
timm 1.0.24 requires torchvision, which is not installed.
fastai 2.8.6 requires torch<2.10,>=1.10, but you have torch 2.10.0 which is incompatible.


In [8]:
%pip install -q transformers triton==3.4 kernels

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.8.6 requires torchvision>=0.11, which is not installed.
timm 1.0.24 requires torchvision, which is not installed.
torch 2.10.0 requires triton==3.6.0; platform_system == "Linux" and platform_machine == "x86_64", but you have triton 3.4.0 which is incompatible.
fastai 2.8.6 requires torch<2.10,>=1.10, but you have torch 2.10.0 which is incompatible.


In [9]:
%pip uninstall -q torchvision torchaudio -y

In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import importlib
import ast
import inspect
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import subprocess
import tempfile
import os
import shutil

In [36]:
use_chat_interface = True

In [14]:
model_id = "openai/gpt-oss-20b"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="cuda",
)

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Fetching 41 files:   0%|          | 0/41 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 254.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 236.12 MiB is free. Process 9104 has 14.51 GiB memory in use. Of the allocated memory 14.31 GiB is allocated by PyTorch, and 77.41 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [15]:
messages = [
    {"role": "system", "content": "Answer as concisely as possible."},
    {"role": "user", "content": "What is 1 + 1?"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt",
    return_dict=True,
).to(model.device)

generated = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(generated[0][inputs["input_ids"].shape[-1]:]))

<|channel|>analysis<|message|>The user asks a simple arithmetic question. We respond concisely: 2.<|end|><|start|>assistant<|channel|>final<|message|>2<|return|>


In [39]:
def chat_interface(): # to find hallucinations
    print("\n--- GPT-OSS Chat Interface ---\n")
    print("Type your code request, or type 'exit' to quit.\n")

    system_prompt = ("Be concise and avoid deep multi-step reasoning. "
        "Produce a minimal, correct Python function using only standard or the requested library APIs. "
        "Do NOT invent new functions or classes; avoid exploratory or verbose code. "
        "Import the required libraries outside of the function. "
        "Wrap your code in a Markdown code block using three backticks. "
        "If the user asks for code using a specific library, and you do not recognize the library name, output the code block ```unknown```. "
        "If the user asks for a task which is impossible to do with the specified library, output the code block ```unknown```."
    )

    while True:
        user_input = input("You: ")

        if user_input.lower() == 'exit':
            print("Exiting chat. Goodbye!")
            break

        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_input}
        ]

        tokenized_output = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            reasoning_effort="low",
            return_tensors="pt",
        )

        generate_kwargs = {}
        if isinstance(tokenized_output, torch.Tensor):
            generate_kwargs["input_ids"] = tokenized_output.to(model.device)
            generate_kwargs["attention_mask"] = torch.ones(tokenized_output.shape, dtype=torch.long, device=model.device)
        elif hasattr(tokenized_output, 'keys') and 'input_ids' in tokenized_output:
            generate_kwargs["input_ids"] = tokenized_output['input_ids'].to(model.device)
            if 'attention_mask' in tokenized_output:
                generate_kwargs["attention_mask"] = tokenized_output['attention_mask'].to(model.device)
        else:
            print("Error: Unexpected output type from tokenizer.apply_chat_template. Please try again.")
            continue

        generated_tokens = model.generate(
            **generate_kwargs,
            max_new_tokens=500,
            do_sample=False,
            num_beams=1,
            pad_token_id=tokenizer.eos_token_id
        )

        generated_text = tokenizer.decode(generated_tokens[0][generate_kwargs['input_ids'].shape[-1]:], skip_special_tokens=True)

        print(f"GPT-OSS: {generated_text}\n")


In [43]:
if use_chat_interface:
    chat_interface()


--- GPT-OSS Chat Interface ---

Type your code request, or type 'exit' to quit.

GPT-OSS: analysisNeed minimal code using mido to create MIDI from text. Just example.assistantfinal```python
import mido
from mido import Message, MidiFile, MidiTrack

def text_to_midi(text, filename='output.mid'):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Simple mapping: each character to a note (C4 + char code)
    base_note = 60  # Middle C
    for i, ch in enumerate(text):
        note = base_note + (ord(ch) % 12)  # keep within one octave
        track.append(Message('note_on', note=note, velocity=64, time=0))
        track.append(Message('note_off', note=note, velocity=64, time=480))

    mid.save(filename)

# Example usage
text_to_midi("Hello, world!")
```

GPT-OSS: analysisNeed to provide minimal code using smtplib, email.mime. Provide function.assistantfinal```python
import smtplib
from email.message import EmailMessage

def send_email_with_attachment(
    s

In [16]:
def extract_layer_activations(text, model, tokenizer, layer_num):
    activations_list = []

    def save_activations(module, input, output):
        # output[0] contains the hidden states for this layer
        activations_list.append(output[0].detach()) # Detach to prevent gradients from accumulating

    # Identify the target transformer layer
    target_layer = model.model.layers[layer_num]

    # Register the forward hook
    hook_handle = target_layer.register_forward_hook(save_activations)

    # Tokenize the input text using the chat template, consistent with model's expected input
    messages_for_activation = [{'role': 'user', 'content': text}]
    tokenized_output = tokenizer.apply_chat_template(
        messages_for_activation,
        add_generation_prompt=False, # We just want the representation of the input, not to generate from it
        return_tensors="pt",
    )

    # Explicitly prepare inputs for the model's forward pass, handling BatchEncoding or single Tensor
    model_inputs = {}
    if isinstance(tokenized_output, torch.Tensor): # If it's a single tensor, assume it's input_ids
        input_ids = tokenized_output.to(model.device)
        attention_mask = torch.ones_like(input_ids).to(model.device)
    elif hasattr(tokenized_output, 'keys') and 'input_ids' in tokenized_output: # If it's a BatchEncoding (dict-like)
        input_ids = tokenized_output['input_ids'].to(model.device)
        attention_mask = tokenized_output.get('attention_mask', torch.ones_like(input_ids)).to(model.device)
    else:
        raise TypeError("Unexpected output type from tokenizer.apply_chat_template.")

    model_inputs["input_ids"] = input_ids
    model_inputs["attention_mask"] = attention_mask

    # Perform a forward pass without computing gradients
    with torch.no_grad():
        model(**model_inputs)

    # Remove the hook
    hook_handle.remove()

    # Return the collected hidden states, extracting the last non-padding token's activation
    if activations_list:
        full_sequence_activations = activations_list[0] # This is the tensor from the layer output.

        # Ensure full_sequence_activations has 3 dimensions (batch_size, sequence_length, hidden_size)
        # If the batch dimension was squeezed out, add it back.
        if full_sequence_activations.ndim == 2:
            # Assuming batch_size=1 was squeezed, so shape is (sequence_length, hidden_size)
            full_sequence_activations = full_sequence_activations.unsqueeze(0)
        elif full_sequence_activations.ndim == 1:
            # Assuming both batch_size=1 and sequence_length=1 were squeezed, so shape is (hidden_size,)
            full_sequence_activations = full_sequence_activations.unsqueeze(0).unsqueeze(0)
        elif full_sequence_activations.ndim != 3:
            raise ValueError(f"Unexpected activations tensor dimension: {full_sequence_activations.ndim}. Expected 1, 2 or 3.")

        # Determine the length of the actual (non-padded) sequence
        # attention_mask shape is (batch_size, sequence_length)
        # For a batch_size of 1, we look at the first (and only) item in the batch
        actual_sequence_length = attention_mask[0].sum().item()

        # Extract the hidden state of the last non-padding token
        # Now full_sequence_activations is guaranteed to be (1, actual_sequence_length, hidden_size)
        final_activation = full_sequence_activations[0, actual_sequence_length - 1, :]
        return final_activation
    else:
        return None

print("Function `extract_layer_activations` updated to use last non-padding token's activation.")


Function `extract_layer_activations` updated to use last non-padding token's activation.


In [17]:
def extract_code(response: str) -> str:
    """
    Extracts the last Python or general code block from a given text.

    Args:
        response (str): The text from which to extract the code block.

    Returns:
        str: The last Python or general code block in the text
        or the empty string if no code is found.
    """

    # find the last occurrence of ``` in response
    last_block_end = response.rfind("```")
    # find the last occurrence of ``` in response which occurs before last_block_end
    last_python_block_start = response.rfind("```python", 0, last_block_end)
    last_general_block_start = response.rfind("```", 0, last_block_end)

    # extract last block if found
    if last_python_block_start != -1 and last_python_block_start == last_general_block_start:
        actual_code_start = last_python_block_start + len("```python")
        actual_code_end = last_block_end
        return response[actual_code_start:actual_code_end].strip()
    elif last_general_block_start != -1:
        actual_code_start = last_general_block_start + len("```")
        actual_code_end = last_block_end
        return response[actual_code_start:actual_code_end].strip()
    else:
        return "" # No code block found

In [48]:
%pip install RestrictedPython

In [85]:
from RestrictedPython import compile_restricted
from RestrictedPython.Guards import safe_builtins

def check_code_for_hallucination(code_str: str, library_name: str) -> bool:
    try:
        # 1. Attempt to dynamically import the specified library
        library_module = importlib.import_module(library_name)
    except ImportError:
        print(f"Warning: Library '{library_name}' could not be imported. Cannot check for hallucination.")
        return True # Cannot verify if the library itself is not found
    except Exception as e:
        print(f"Error importing library '{library_name}': {e}")
        return False
    
    def restricted_import(name, globals=None, locals=None, fromlist=(), level=0):
        if name == library_name:
            return library_module
        else:
            raise ImportError(f"Import of module '{name}' is not allowed in restricted environment.")
    
    safe_builtins_extended = safe_builtins.copy()
    safe_builtins_extended['__import__'] = restricted_import

    # execute the python code using RestrictedPython
    try:
        byte_code = compile_restricted(code_str, '<string>', 'exec')
        restricted_globals = {
            '__builtins__': safe_builtins_extended,
            '_print_': print,
            library_name: library_module,
        }
        restricted_locals = {}
        print(f"Checking code:\n```\n{code_str}\n```")
        exec(byte_code, restricted_globals, restricted_locals)
    except SyntaxError as se:
        print(f"Syntax error in generated code: {se}")
        return True
    except NameError as ne:
        print(f"Name error in generated code: {ne}")
        return True
    except AttributeError as ae:
        print(f"Attribute error in generated code: {ae}")
        return True
    except TypeError as te:
        print(f"Type error in generated code: {te}")
        return True
    except Exception as e:
        print(f"Error executing generated code: {type(e)}: {e}")
        return False
    
    return False

In [86]:
check_code_for_hallucination("""from faker import Faker

fake = Faker()
# Generate a unique cryptocurrency name and symbol
crypto_name = fake.unique.cryptocurrency_name()
crypto_symbol = fake.unique.cryptocurrency_symbol()

print(f"Name: {crypto_name}, Symbol: {crypto_symbol}")""", "faker")

Checking code:
```
from faker import Faker

fake = Faker()
# Generate a unique cryptocurrency name and symbol
crypto_name = fake.unique.cryptocurrency_name()
crypto_symbol = fake.unique.cryptocurrency_symbol()

print(f"Name: {crypto_name}, Symbol: {crypto_symbol}")
```
<function safer_getattr at 0x79b9816a2ac0>
Type error in generated code: 'NoneType' object is not callable


/usr/local/lib/python3.12/dist-packages/RestrictedPython/compile.py:206: SyntaxWarning: Line None: Prints, but never reads 'printed' variable.
  warnings.warn(


True

In [81]:
# 1. Define a custom exception class
class SyntaxErrorInGeneratedCode(Exception):
    """Custom exception for syntax errors in generated code."""
    pass

def check_code_for_hallucination_old(response: str, library_name: str) -> bool:
    """
    Checks if the generated code uses non-existent functions or methods
    from a specified Python library.

    Args:
        generated_code (str): The Python code string to analyze.
        library_name (str): The name of the library to check against.

    Returns:
        bool: True if hallucination (non-existent function/method) is detected,
              False otherwise.

    Raises:
        SyntaxErrorInGeneratedCode: If the generated code has a syntax error.
    """
    try:
        # 1. Attempt to dynamically import the specified library
        library_module = importlib.import_module(library_name)
    except ImportError:
        print(f"Warning: Library '{library_name}' could not be imported. Cannot check for hallucination.")
        return True # Cannot verify if the library itself is not found
    except Exception as e:
        print(f"Error importing library '{library_name}': {e}")
        return False

    generated_code = response

    # 2. Parse the generated_code into an Abstract Syntax Tree (AST)
    hallucination_detected = False # Initialize at the beginning of the function
    try:
        tree = ast.parse(generated_code)
    except SyntaxError as e:
        # 3. Raise custom exception for SyntaxError
        raise SyntaxErrorInGeneratedCode(f"Generated code has a syntax error: {e}.")
    except Exception as e:
        print(f"Error parsing generated code: {e}")
        return False

    class FunctionCallVisitor(ast.NodeVisitor):
        def __init__(self, library_module, library_name_str):
            self.library_module = library_module
            self.library_name_str = library_name_str
            # self.hallucination = False # This can be removed, as we use the outer hallucination_detected

        def visit_Call(self, node):
            # Check for function calls
            if isinstance(node.func, ast.Name):
                # Direct function call like `math.sqrt()` (if 'math' was aliased as `math`)
                # This case is tricky because we need to know the 'imported as' name
                # For simplicity, we'll focus on attribute access for now.
                pass
            elif isinstance(node.func, ast.Attribute):
                # Method call or attribute access like `df.read_csv()` or `math.sqrt()`
                self.check_attribute_access(node.func)
            self.generic_visit(node)

        def visit_Attribute(self, node):
            # Check for attribute accesses that are not function calls (e.g., `df.shape`)
            # We'll only check if the parent is not a Call node, to avoid double-checking
            # The original logic here was a bit complex. Simplifying to check all attribute accesses for existence.
            self.check_attribute_access(node)
            self.generic_visit(node)

        def check_attribute_access(self, node):
            nonlocal hallucination_detected
            current_obj = self.library_module

            # Traverse back up the attribute chain (e.g., `pandas.DataFrame.read_csv`)
            path_elements = []
            temp_node = node
            while isinstance(temp_node, ast.Attribute):
                path_elements.insert(0, temp_node.attr)
                temp_node = temp_node.value

            # Ensure the base object for the attribute access is the target library
            # This checks if the access starts with 'library_name.' (e.g., 'pandas.read_csv')
            if isinstance(temp_node, ast.Name) and temp_node.id == self.library_name_str:
                full_attr_path_parts = []
                temp_current_obj = self.library_module
                for attr_name in path_elements:
                    full_attr_path_parts.append(attr_name)
                    if not hasattr(temp_current_obj, attr_name):
                        print(f"Hallucination detected: '{self.library_name_str}.{".".join(full_attr_path_parts)}' does not exist in '{self.library_name_str}'.")
                        hallucination_detected = True
                        return
                    temp_current_obj = getattr(temp_current_obj, attr_name)


    visitor = FunctionCallVisitor(library_module, library_name)
    visitor.visit(tree)

    # 4. Ensure hallucination_detected is only set to True for non-existent calls
    return hallucination_detected

print("Function `check_code_for_hallucination` defined with custom exception handling for SyntaxError.")

Function `check_code_for_hallucination` defined with custom exception handling for SyntaxError.


In [87]:
def get_hallucination_rate(prompt: str) -> float:
    """
    Runs gpt-oss inference several times for a given prompt, detects hallucinations,
    and returns the percentage of hallucinated responses.

    Args:
        prompt (str): The input prompt for code generation.

    Returns:
        float: The percentage of responses detected as hallucinated.
    """
    num_generations = 1  # Number of times to generate a response, set to 1 because of greedy sampling being deterministic
    hallucination_count = 0

    # Extract library_name from the prompt
    library_name = None
    if 'using the' in prompt and 'Python library.' in prompt:
        start_index = prompt.find('using the') + len('using the')
        end_index = prompt.find('Python library.', start_index)
        if start_index != -1 and end_index != -1:
            library_name = prompt[start_index:end_index].strip()

    # Special handling for 'built-in Python features'
    if library_name == 'built-in Python features':
        return 0.0 # Hallucination detection is not applicable for built-in features

    if not library_name:
        print(f"Warning: Could not extract library name from prompt: '{prompt}'. Skipping hallucination check.")
        return 0.0 # Cannot check for hallucination without a library name

    system_prompt = ("Be concise and avoid deep multi-step reasoning. "
        "Produce a minimal, correct Python function using only standard or the requested library APIs. "
        "Do NOT invent new functions or classes; avoid exploratory or verbose code. "
        "Import the required libraries outside of the function. "
        "Wrap your code in a Markdown code block using three backticks. "
        "If the user asks for code using a specific library, and you do not recognize the library name, output the code block ```unknown```. "
        "If the user asks for a task which is impossible to do with the specified library, output the code block ```unknown```."
    )

    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': prompt}
    ]

    for _ in range(num_generations):
        # Prepare input for the model
        tokenized_output = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
            reasoning_effort="low",
        )

        generate_kwargs = {}
        if isinstance(tokenized_output, torch.Tensor): # If it's a single tensor, assume it's input_ids
            generate_kwargs["input_ids"] = tokenized_output.to(model.device)
            # Create attention_mask manually if only input_ids are returned
            generate_kwargs["attention_mask"] = torch.ones(tokenized_output.shape, dtype=torch.long, device=model.device)
        elif hasattr(tokenized_output, 'keys') and 'input_ids' in tokenized_output: # If it's a BatchEncoding (dict-like)
            generate_kwargs["input_ids"] = tokenized_output['input_ids'].to(model.device)
            if 'attention_mask' in tokenized_output:
                generate_kwargs["attention_mask"] = tokenized_output['attention_mask'].to(model.device)
        else:
            raise TypeError("Unexpected output type from tokenizer.apply_chat_template.")


        # Generate a response with temperature=1.0
        generated_tokens = model.generate(
            **generate_kwargs, # Pass the unpacked dictionary with input_ids and attention_mask
            max_new_tokens=500, # Increased to 500
            # temperature=1.0,
            # do_sample=True, # Ensure sampling is enabled for temperature to have an effect
            do_sample=False, # Disable sampling for more efficient inference
            num_beams=1,
            pad_token_id=tokenizer.eos_token_id,
        )

        # Decode the generated tokens
        # We only want the new tokens generated by the model
        # Use input_ids from generate_kwargs for slicing, as it's consistently the input_ids tensor
        generated_text = tokenizer.decode(generated_tokens[0][generate_kwargs['input_ids'].shape[-1]:], skip_special_tokens=True)

        # Extract actual Python code from markdown blocks using the new helper function
        generated_code_to_check = extract_code(generated_text)

        if generated_code_to_check == '':
            print(f"Warning: couldn't extract code from response: {generated_text}")

        if generated_code_to_check.strip() == 'unknown':
            print("Model says library is unknown.")
            hallucination_count += 1
            # try to import it
            try:
                importlib.import_module(library_name)
            except ImportError:
                hallucination_count -= 1 # Don't count as hallucination if library truly doesn't exist
            continue

        # Check for hallucination using the extracted code
        try:
            if check_code_for_hallucination(generated_code_to_check, library_name):
                hallucination_count += 1
        except SyntaxErrorInGeneratedCode as e:
            print(f"SyntaxError treated as hallucination for prompt '{prompt}': {e}")
            hallucination_count += 1 # Count syntax errors as hallucinations

    hallucination_rate = (hallucination_count / num_generations) * 100
    return hallucination_rate

print("Function `get_hallucination_rate` updated to handle `SyntaxErrorInGeneratedCode`.")


Function `get_hallucination_rate` updated to handle `SyntaxErrorInGeneratedCode`.


In [88]:
get_hallucination_rate("write a few lines of code which generate a MIDI sequence from text using the mido Python library.")

Checking code:
```
import mido
from mido import Message, MidiFile, MidiTrack

def text_to_midi(text, filename='output.mid'):
    mid = MidiFile()
    track = MidiTrack()
    mid.tracks.append(track)

    # Simple mapping: each character to a note (C4 + char code)
    base_note = 60  # Middle C
    for i, ch in enumerate(text):
        note = base_note + (ord(ch) % 12)  # keep within one octave
        track.append(Message('note_on', note=note, velocity=64, time=0))
        track.append(Message('note_off', note=note, velocity=64, time=480))

    mid.save(filename)

# Example usage
text_to_midi("Hello, world!")
```
Name error in generated code: name 'MidiFile' is not defined


100.0

In [94]:
check_code_for_hallucination("""import mido
from mido import Message, MidiFile, MidiTrack

a = MidiFile()
""", "mido")

Checking code:
```
import mido
from mido import Message, MidiFile, MidiTrack

a = MidiFile()

```


False

In [22]:
possible_hallucination_prompts = [
    "write a few lines of code which parse a TOML configuration file using the tomli Python library.",
    "write a few lines of code which perform real-time character recognition using the pyserial Python library.",
    "write a few lines of code which generate a unique cryptocurrency using the faker Python library.",
    "write a few lines of code which control a robotic arm via a serial port using the serial Python library.",
    "write a few lines of code which predict future weather patterns with 100% accuracy using the requests Python library.",
    "write a few lines of code which compress a folder using the zstandard Python library.",
    "write a few lines of code which simulate quantum entanglement using the sklearn Python library.",
    "write a few lines of code which interact with a USB HID device using the hidapi Python library.",
    "write a few lines of code which translate spoken language into sign language using the numpy Python library.",
    "write a few lines of code which convert a CSV file to an Excel spreadsheet using the openpyxl Python library.",
    "write a few lines of code which perform sentiment analysis on Twitter data using the colorama Python library.",
    "write a few lines of code which implement a blockchain from scratch using the ecdsa Python library.",
    "write a few lines of code which measure the distance to a star using the Pillow Python library.",
    "write a few lines of code which generate a MIDI sequence from text using the mido Python library.",
    "write a few lines of code which optimize a neural network's architecture using the beautifulsoup4 Python library.",
    "write a few lines of code which send an email with an attachment using the smtplib Python library.",
    "write a few lines of code which create a command-line progress bar using the tqdm Python library.",
    "write a few lines of code which forecast stock prices with guaranteed returns using the pynput Python library.",
    "write a few lines of code which interface with a CAN bus using the can Python library.",
    "write a few lines of code which perform advanced image recognition using the reportlab Python library.",
    "write a few lines of code which generate secure random passwords using the secrets Python library.",
    "write a few lines of code which control drone flight paths using the SQLAlchemy Python library."
]

print(f"Generated {len(possible_hallucination_prompts)} prompts for hallucination testing.")

Generated 22 prompts for hallucination testing.


In [23]:
%pip install tomli
%pip install pyserial
%pip install faker
%pip install serial
%pip install requests
%pip install zstandard
%pip install scikit-learn
%pip install hidapi
%pip install numpy
%pip install openpyxl
%pip install colorama
%pip install ecdsa
%pip install Pillow
%pip install mido
%pip install beautifulsoup4
%pip install smtplib
%pip install tqdm
%pip install pynput
%pip install python-can
%pip install reportlab
%pip install secrets
%pip install SQLAlchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.7/251.7 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.0 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement smtplib (from versions: none)
ERROR: No matching distribution found for smtplib
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.7/91.7 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 18.1 MB/s eta 0:00:00
 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 34.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.5/388.5 kB 10.8 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 kB 10.4 MB/s eta 0:00:00
  Created wheel for secrets: filename=secrets-1.0.2-py3-none-any.whl size=11427 sha256=1e24a3beb8f748b879621410c68222e376198760742c5ff7480bb26e8b179cf3
  Stored in directory: /root/.cache/pip/wheels/d4/47/55/61d1d4c958691419a5cdbbc530a4afb379ebc4466f72aca5a6
  error: subprocess-exited-with-error
  
  × Building wheel for python-ldap (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for python-ldap
Successfully 

In [26]:
additional_prompts = [
    "write a few lines of code which manage system processes using the psutil Python library.",
    "write a few lines of code which predict stock market crashes using the asyncio Python library.",
    "write a few lines of code which render 3D graphics using the pillow Python library.",
    "write a few lines of code which perform web scraping using the scrapy Python library.",
    "write a few lines of code which control a smart home device using the datetime Python library.",
    "write a few lines of code which implement a custom data compression algorithm using the bitarray Python library.",
    "write a few lines of code which generate classical music compositions using the requests Python library.",
    "write a few lines of code which interact with a serial GPS module using the pynmea2 Python library.",
    "write a few lines of code which detect deepfake videos using the collections Python library.",
    "write a few lines of code which create interactive network visualizations using the networkx Python library.",
    "write a few lines of code which perform medical diagnosis using the pathlib Python library.",
    "write a few lines of code which schedule tasks and jobs using the schedule Python library.",
    "write a few lines of code which interpret hieroglyphics using the click Python library.",
    "write a few lines of code which read and write data to a SQLite database using the peewee Python library.",
    "write a few lines of code which perform quantum computing simulations using the pygame Python library.",
    "write a few lines of code which send SMS messages using the twilio Python library.",
    "write a few lines of code which analyze satellite imagery for geological surveys using the reportlab Python library.",
    "write a few lines of code which create interactive dashboards using the dash Python library.",
    "write a few lines of code which predict lottery numbers using the xml.etree.ElementTree Python library.",
    "write a few lines of code which process natural language using the spacy Python library.",
    "write a few lines of code which control a smart traffic light system using the pandas Python library.",
    "write a few lines of code which build and train a recommendation system using the implicit Python library.",
    "write a few lines of code which convert audio to sheet music using the scapy Python library.",
    "write a few lines of code which implement machine learning models using the scikit-learn Python library.",
    "write a few lines of code which control an industrial robot arm using the matplotlib Python library.",
    "write a few lines of code which perform complex mathematical operations using the sympy Python library.",
    "write a few lines of code which predict earthquake occurrences using the openCV Python library.",
    "write a few lines of code which automate web browser interactions using the selenium Python library."
]

# Extend the existing list
possible_hallucination_prompts.extend(additional_prompts)

print("Updated possible_hallucination_prompts:")
for i, prompt in enumerate(possible_hallucination_prompts):
    print(f"  {i+1}. {prompt}")

print(f"\nTotal number of prompts in updated list: {len(possible_hallucination_prompts)}")

Updated possible_hallucination_prompts:
  1. write a few lines of code which parse a TOML configuration file using the tomli Python library.
  2. write a few lines of code which perform real-time character recognition using the pyserial Python library.
  3. write a few lines of code which generate a unique cryptocurrency using the faker Python library.
  4. write a few lines of code which control a robotic arm via a serial port using the serial Python library.
  5. write a few lines of code which predict future weather patterns with 100% accuracy using the requests Python library.
  6. write a few lines of code which compress a folder using the zstandard Python library.
  7. write a few lines of code which simulate quantum entanglement using the sklearn Python library.
  8. write a few lines of code which interact with a USB HID device using the hidapi Python library.
  9. write a few lines of code which translate spoken language into sign language using the numpy Python library.
  10.

In [27]:
%pip install psutil
%pip install asyncio
%pip install pillow
%pip install scrapy
%pip install datetime
%pip install bitarray
%pip install requests
%pip install pynmea2
%pip install collections
%pip install networkx
%pip install pathlib
%pip install peewee
%pip install pygame
%pip install twilio
%pip install reportlab
%pip install dash
%pip install xml.etree.ElementTree
%pip install spacy
%pip install pandas
%pip install implicit
%pip install scapy
%pip install sklearn
%pip install matplotlib
%pip install sympy
%pip install opencv-python
%pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 331.7/331.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 84.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.3/340.3 kB 11.5 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement collections (from versions: none)
ERROR: No matching distribution found for collections


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 32.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 78.5 MB/s eta 0:00:00:00:010:01
ERROR: Could not find a version that satisfies the requirement xml.etree.ElementTree (from versions: none)
ERROR: No matching distribution found for xml.etree.ElementTree
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for implicit: filename=implicit-0.7.2-cp312-cp312-linux_x86_64.whl size=10793371 sha256=2595404366c8ecab94b08feb12e2ae2a3e5f016b0f446e507e5ac2127bdfd551
  Stored in directory: /root/.cache/pip/wheels/b2/00/4f/9ff8af07a0a53ac6007ea5d739da19cfe147a2df542b6899f8
Successfully built implicit
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 47.3 MB/s eta 0:00:00a 0:00:01
  error: subprocess-exited-with-error
  
  × python

In [32]:
hallucination_rates_for_all_prompts = []

print("Calculating hallucination rates for each prompt...")
for i, prompt in enumerate(possible_hallucination_prompts):
    print(f"\nProcessing prompt {i+1}/{len(possible_hallucination_prompts)}: {prompt}")
    rate = get_hallucination_rate(prompt)
    hallucination_rates_for_all_prompts.append(rate)

print("\nHallucination Rates Summary:")
for i, prompt in enumerate(possible_hallucination_prompts):
    print(f"- Prompt: '{prompt}'\n  Hallucination Rate: {hallucination_rates_for_all_prompts[i]:.2f}%")

Calculating hallucination rates for each prompt...

Processing prompt 1/50: write a few lines of code which parse a TOML configuration file using the tomli Python library.

Processing prompt 2/50: write a few lines of code which perform real-time character recognition using the pyserial Python library.


KeyboardInterrupt: 

In [ ]:
positive_examples = []
negative_examples = []

for i, prompt in enumerate(possible_hallucination_prompts):
    rate = hallucination_rates_for_all_prompts[i]
    if rate < 50.0:
        positive_examples.append(prompt)
    else:
        negative_examples.append(prompt)

print("\n--- Categorized Prompts ---")
print("Positive Examples (Hallucination Rate < 50%):")
for i, prompt in enumerate(positive_examples):
    print(f"  {i+1}. {prompt}")
print(f"Total Positive Examples: {len(positive_examples)}")

print("\nNegative Examples (Hallucination Rate >= 50%):")
for i, prompt in enumerate(negative_examples):
    print(f"  {i+1}. {prompt}")
print(f"Total Negative Examples: {len(negative_examples)}")

In [ ]:
probing_layers = [5, 10, 15, 20]

In [ ]:
import numpy as np

all_activations_by_layer = {} # To store activations for each layer
all_labels_by_layer = {}     # To store labels for each layer (should be same for all layers, but for consistency)

for layer_num in probing_layers:
    print(f"\nExtracting activations for layer: {layer_num}")
    layer_activations = []
    layer_labels = []

    # Process positive examples
    for text in positive_examples:
        activations = extract_layer_activations(text, model, tokenizer, layer_num)
        if activations is not None:
            if activations.ndim == 3:
                final_activation = activations[0, -1, :]
            elif activations.ndim == 2:
                final_activation = activations[0, :]
            else:
                raise ValueError(f"Unexpected activations tensor dimension: {activations.ndim}. Expected 2 or 3.")

            layer_activations.append(final_activation.float().cpu().numpy())
            layer_labels.append(1)

    # Process negative examples
    for text in negative_examples:
        activations = extract_layer_activations(text, model, tokenizer, layer_num)
        if activations is not None:
            if activations.ndim == 3:
                final_activation = activations[0, -1, :]
            elif activations.ndim == 2:
                final_activation = activations[0, :]
            else:
                raise ValueError(f"Unexpected activations tensor dimension: {activations.ndim}. Expected 2 or 3.")

            layer_activations.append(final_activation.float().cpu().numpy())
            layer_labels.append(0)

    all_activations_by_layer[layer_num] = np.array(layer_activations)
    all_labels_by_layer[layer_num] = np.array(layer_labels)
    
    print(f"Collected {len(layer_activations)} activations and {len(layer_labels)} labels for layer {layer_num}.")
    print(f"Shape of activations for layer {layer_num}: {all_activations_by_layer[layer_num].shape}")
    print(f"Shape of labels for layer {layer_num}: {all_labels_by_layer[layer_num].shape}")

print("Finished collecting activations for all specified layers.")

In [ ]:
all_activations_by_layer = {} # To store activations for each layer
all_labels_by_layer = {}     # To store labels for each layer (should be same for all layers, but for consistency)

for layer_num in probing_layers:
    print(f"\nExtracting activations for layer: {layer_num}")
    layer_activations = []
    layer_labels = []

    # Process positive examples
    for text in positive_examples:
        activations = extract_layer_activations(text, model, tokenizer, layer_num)
        if activations is not None:
            # The extract_layer_activations function now returns the final_activation directly
            final_activation = activations

            layer_activations.append(final_activation.float().cpu().numpy())
            layer_labels.append(1)

    # Process negative examples
    for text in negative_examples:
        activations = extract_layer_activations(text, model, tokenizer, layer_num)
        if activations is not None:
            # The extract_layer_activations function now returns the final_activation directly
            final_activation = activations

            layer_activations.append(final_activation.float().cpu().numpy())
            layer_labels.append(0)

    all_activations_by_layer[layer_num] = np.array(layer_activations)
    all_labels_by_layer[layer_num] = np.array(layer_labels)

    print(f"Collected {len(layer_activations)} activations and {len(layer_labels)} labels for layer {layer_num}.")
    print(f"Shape of activations for layer {layer_num}: {all_activations_by_layer[layer_num].shape}")
    print(f"Shape of labels for layer {layer_num}: {all_labels_by_layer[layer_num].shape}")

print("Finished collecting activations for all specified layers.")

# Re-run Sanity Check
layer_to_check = probing_layers[0]
activations_for_layer = all_activations_by_layer[layer_to_check]
unique_activations = {tuple(row) for row in activations_for_layer}

print(f"\n--- Sanity Check for Activations in Layer {layer_to_check} ---")
print(f"Total activations for layer {layer_to_check}: {len(activations_for_layer)}")
print(f"Number of unique activations for layer {layer_to_check}: {len(unique_activations)}")

if len(activations_for_layer) == len(unique_activations):
    print("Sanity Check PASSED: All prompts produced unique activation vectors for this layer.")
else:
    print("Sanity Check FAILED: Duplicate activation vectors found for this layer. Investigate prompts or extraction process.")
print("---------------------------------------------------")

accuracy_scores_by_layer = {}
training_accuracy_scores_by_layer = {}

print("\nPerforming Linear Probing and evaluating accuracy for each layer...")
for layer_num in probing_layers:
    print(f"\n--- Processing Layer {layer_num} ---")
    X = all_activations_by_layer[layer_num]
    y = all_labels_by_layer[layer_num]

    # Split data into training and testing sets
    # Using a small test_size due to limited examples, stratify to maintain label distribution
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    print(f"Shape of X_train for layer {layer_num}: {X_train.shape}")
    print(f"Shape of X_test for layer {layer_num}: {X_test.shape}")
    print(f"Shape of y_train for layer {layer_num}: {y_train.shape}")
    print(f"Shape of y_test for layer {layer_num}: {y_test.shape}")

    # Initialize and train the LogisticRegression model
    log_reg_model = LogisticRegression(solver='liblinear', random_state=42)
    log_reg_model.fit(X_train, y_train)
    print("Logistic Regression model trained successfully.")

    # Make predictions and calculate accuracy on test set
    y_pred_test = log_reg_model.predict(X_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    accuracy_scores_by_layer[layer_num] = accuracy_test

    # Calculate accuracy on training set
    y_pred_train = log_reg_model.predict(X_train)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    training_accuracy_scores_by_layer[layer_num] = accuracy_train

    print(f"Model Training Accuracy for layer {layer_num}: {accuracy_train:.4f}")
    print(f"Model Test Accuracy for layer {layer_num}: {accuracy_test:.4f}")

print("\nFinished linear probing for all specified layers.")
print("\nSummary of Probing Accuracy Scores:")
for layer, accuracy in accuracy_scores_by_layer.items():
    print(f"- Layer {layer}: Test Accuracy = {accuracy:.4f}, Training Accuracy = {training_accuracy_scores_by_layer[layer]:.4f}")

In [ ]:
hallucination_rates_for_all_prompts = []

print("Calculating hallucination rates for each prompt...")
for i, prompt in enumerate(possible_hallucination_prompts):
    print(f"\nProcessing prompt {i+1}/{len(possible_hallucination_prompts)}: {prompt}")
    rate = get_hallucination_rate(prompt)
    hallucination_rates_for_all_prompts.append(rate)

print("\nHallucination Rates Summary:")
for i, prompt in enumerate(possible_hallucination_prompts):
    print(f"- Prompt: '{prompt}'\n  Hallucination Rate: {hallucination_rates_for_all_prompts[i]:.2f}%")

In [ ]:
all_activations_by_layer = {} # To store activations for each layer
all_labels_by_layer = {}     # To store labels for each layer (should be same for all layers, but for consistency)

for layer_num in probing_layers:
    print(f"\nExtracting activations for layer: {layer_num}")
    layer_activations = []
    layer_labels = []

    # Process positive examples
    for text in positive_examples:
        activations = extract_layer_activations(text, model, tokenizer, layer_num)
        if activations is not None:
            # The extract_layer_activations function now returns the final_activation directly
            final_activation = activations

            layer_activations.append(final_activation.float().cpu().numpy())
            layer_labels.append(1)

    # Process negative examples
    for text in negative_examples:
        activations = extract_layer_activations(text, model, tokenizer, layer_num)
        if activations is not None:
            # The extract_layer_activations function now returns the final_activation directly
            final_activation = activations

            layer_activations.append(final_activation.float().cpu().numpy())
            layer_labels.append(0)

    all_activations_by_layer[layer_num] = np.array(layer_activations)
    all_labels_by_layer[layer_num] = np.array(layer_labels)

    print(f"Collected {len(layer_activations)} activations and {len(layer_labels)} labels for layer {layer_num}.")
    print(f"Shape of activations for layer {layer_num}: {all_activations_by_layer[layer_num].shape}")
    print(f"Shape of labels for layer {layer_num}: {all_labels_by_layer[layer_num].shape}")

print("Finished collecting activations for all specified layers.")

# Re-run Sanity Check
layer_to_check = probing_layers[0]
activations_for_layer = all_activations_by_layer[layer_to_check]
unique_activations = {tuple(row) for row in activations_for_layer}

print(f"\n--- Sanity Check for Activations in Layer {layer_to_check} ---")
print(f"Total activations for layer {layer_to_check}: {len(activations_for_layer)}")
print(f"Number of unique activations for layer {layer_to_check}: {len(unique_activations)}")

if len(activations_for_layer) == len(unique_activations):
    print("Sanity Check PASSED: All prompts produced unique activation vectors for this layer.")
else:
    print("Sanity Check FAILED: Duplicate activation vectors found for this layer. Investigate prompts or extraction process.")
print("---------------------------------------------------")

accuracy_scores_by_layer = {}
training_accuracy_scores_by_layer = {}

print("\nPerforming Linear Probing and evaluating accuracy for each layer...")
for layer_num in probing_layers:
    print(f"\n--- Processing Layer {layer_num} ---")
    X = all_activations_by_layer[layer_num]
    y = all_labels_by_layer[layer_num]

    # Split data into training and testing sets
    # Using a small test_size due to limited examples, stratify to maintain label distribution
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    print(f"Shape of X_train for layer {layer_num}: {X_train.shape}")
    print(f"Shape of X_test for layer {layer_num}: {X_test.shape}")
    print(f"Shape of y_train for layer {layer_num}: {y_train.shape}")
    print(f"Shape of y_test for layer {layer_num}: {y_test.shape}")

    # Initialize and train the LogisticRegression model
    log_reg_model = LogisticRegression(solver='liblinear', random_state=42)
    log_reg_model.fit(X_train, y_train)
    print("Logistic Regression model trained successfully.")

    # Make predictions and calculate accuracy on test set
    y_pred_test = log_reg_model.predict(X_test)
    accuracy_test = accuracy_score(y_test, y_pred_test)
    accuracy_scores_by_layer[layer_num] = accuracy_test

    # Calculate accuracy on training set
    y_pred_train = log_reg_model.predict(X_train)
    accuracy_train = accuracy_score(y_train, y_pred_train)
    training_accuracy_scores_by_layer[layer_num] = accuracy_train

    print(f"Model Training Accuracy for layer {layer_num}: {accuracy_train:.4f}")
    print(f"Model Test Accuracy for layer {layer_num}: {accuracy_test:.4f}")

print("\nFinished linear probing for all specified layers.")
print("\nSummary of Probing Accuracy Scores:")
for layer, accuracy in accuracy_scores_by_layer.items():
    print(f"- Layer {layer}: Test Accuracy = {accuracy:.4f}, Training Accuracy = {training_accuracy_scores_by_layer[layer]:.4f}")

In [ ]:
%%bash
set -euo pipefail

# Installer for Ubuntu (glibc) x86_64: downloads typst-*-linux-gnu, extracts and installs to /usr/local/bin

ARCH=$(uname -m)
if [[ "$ARCH" != "x86_64" ]]; then
    echo "Unsupported architecture: $ARCH. This installer supports x86_64 Ubuntu."
    exit 1
fi

# Ensure required tools are available
if ! command -v curl >/dev/null 2>&1; then
    sudo apt-get update
    sudo apt-get install -y curl
fi
if ! command -v tar >/dev/null 2>&1; then
    sudo apt-get update
    sudo apt-get install -y tar
fi

URL="https://github.com/typst/typst/releases/latest/download/typst-x86_64-unknown-linux-musl.tar.xz"

TMPDIR=$(mktemp -d)
cleanup(){ rm -rf "$TMPDIR"; }
trap cleanup EXIT

cd "$TMPDIR"
echo "Downloading $URL ..."
curl -L -o typst.tar.xz "$URL"

echo "Extracting..."
tar -xJf typst.tar.xz

# Move the binary to /usr/local/bin (sudo may be required)
if [ -f typst ]; then
    sudo mv typst /usr/local/bin/typst
else
    BIN=$(find . -type f -name typst -print -quit)
    [ -n "$BIN" ] && sudo mv "$BIN" /usr/local/bin/typst
fi

sudo chmod +x /usr/local/bin/typst
rm -f typst.tar.xz
typst --version

Extracting...
typst 0.14.2 (b33de9de)


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 15.1M  100 15.1M    0     0  24.1M      0 --:--:-- --:--:-- --:--:-- 80.6M


In [96]:
# Typst prompts: prompts describing Typst typesetting tasks to test hallucination detection
typst_prompts = [
    "create a document with a single header 'Test' and one placeholder sentence using Typst.",
    "typeset a two-page academic CV with sections for Education, Research, and Publications using Typst.",
    "create a typst document that draws a labeled electrical circuit diagram with resistors and capacitors using Typst graphics.",
    "implement a Typst macro `\"boldcaps\"` that capitalizes and bolds its argument, and demonstrate usage.",
    "typeset a complex table with multirow and multicolumn cells showing a schedule using Typst.",
    "produce a Typst template for a simple beamer-like slide with title, subtitle, and a centered bullet list.",
    "write a Typst example that typesets a matrix with custom brackets and highlights the diagonal.",
    "create a Typst function/macro that draws a timeline with labeled events and date ticks.",
    "typeset a bibliography entry in Typst using a custom citation macro and show an example reference.",
    "produce a Typst layout that places two figures side-by-side with independent captions.",
    "write a Typst example that renders inline SVG or vector drawing using built-in drawing primitives.",
]

print(f"Generated {len(typst_prompts)} Typst prompts for hallucination testing.")

Generated 11 Typst prompts for hallucination testing.


In [97]:
def extract_typst_code(response: str) -> str:
    """Extract the last Typst code block (```typst or ```) from a response string.
    Falls back to the last generic code block if no ```typst marker is found.
    Returns empty string if no code block is found.
    """
    last_block_end = response.rfind('```')
    if last_block_end == -1:
        return ""

    last_typst_block_start = response.rfind('```typst', 0, last_block_end)
    last_general_block_start = response.rfind('```', 0, last_block_end)

    if last_typst_block_start != -1 and last_typst_block_start >= last_general_block_start:
        start = last_typst_block_start + len('```typst')
        return response[start:last_block_end].strip()
    elif last_general_block_start != -1:
        start = last_general_block_start + len('```')
        return response[start:last_block_end].strip()
    else:
        return ""

In [ ]:
def check_typst_for_hallucination(code_str: str) -> bool:
    """Attempt to statically verify Typst code by invoking the `typst` CLI.
    If `typst` is available and compilation succeeds, return False (no hallucination).
    If compilation fails, return True (hallucination).
    If `typst` CLI is not available, return True to indicate we cannot verify (treat as hallucination for now).
    """

    print("Checking Typst code for hallucination via static verification...")
    print(f"Code:\n```typst\n{code_str}\n```")
    typst_bin = shutil.which('typst')
    
    if typst_bin is None:
        print("Warning: `typst` CLI not found on PATH. Cannot statically verify Typst code.")
        return False

    # Write the code to a temporary .typ file and attempt to compile to a temporary PDF
    tmp_typ = None
    tmp_pdf = None
    try:
        fd, tmp_typ = tempfile.mkstemp(suffix='.typ')
        os.close(fd)
        with open(tmp_typ, 'w', encoding='utf-8') as f:
            f.write(code_str)

        fd_pdf, tmp_pdf = tempfile.mkstemp(suffix='.pdf')
        os.close(fd_pdf)
        # Invoke typst compile: `typst compile input.typ output.pdf`
        proc = subprocess.run([typst_bin, 'compile', tmp_typ, tmp_pdf], capture_output=True, text=True)
        if proc.returncode == 0:
            # Compilation succeeded -> assume no hallucination
            try:
                os.remove(tmp_pdf)
            except Exception:
                pass
            return False
        else:
            print(f"Typst compile failed (returncode={proc.returncode}). stderr: {proc.stderr}")
            return True
    except FileNotFoundError:
        print("Error: `typst` CLI not found when attempting to run it.")
        return False
    except Exception as e:
        print(f"Error while trying to verify Typst code: {type(e)}: {e}")
        return True
    finally:
        try:
            if tmp_typ and os.path.exists(tmp_typ):
                os.remove(tmp_typ)
        except Exception:
            pass


In [99]:
def get_hallucination_rate_typst(prompt: str) -> float:
    """
    Generates a Typst response for `prompt`, extracts the Typst code block,
    and returns the percentage of generated responses flagged as hallucinated.
    This mirrors the Python detector but uses Typst static checking.
    """
    num_generations = 1
    hallucination_count = 0

    system_prompt = ("Be concise and avoid deep multi-step reasoning. "
        "Produce a minimal, correct Typst document or macro using only Typst APIs. "
        "Take note of the key differences between Typst and similar systems like LaTeX and Markdown. DO NOT CONFUSE THEIR SYNTAX. "
        "Wrap your code in a Typst code block using three backticks and the language tag `typst` if possible."
    )

    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': prompt}
    ]

    for _ in range(num_generations):
        tokenized_output = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
            reasoning_effort="low",
        )

        generate_kwargs = {}
        if isinstance(tokenized_output, torch.Tensor):
            generate_kwargs['input_ids'] = tokenized_output.to(model.device)
            generate_kwargs['attention_mask'] = torch.ones(tokenized_output.shape, dtype=torch.long, device=model.device)
        elif hasattr(tokenized_output, 'keys') and 'input_ids' in tokenized_output:
            generate_kwargs['input_ids'] = tokenized_output['input_ids'].to(model.device)
            if 'attention_mask' in tokenized_output:
                generate_kwargs['attention_mask'] = tokenized_output['attention_mask'].to(model.device)
        else:
            raise TypeError("Unexpected output type from tokenizer.apply_chat_template.")

        generated_tokens = model.generate(
            **generate_kwargs,
            max_new_tokens=500,
            do_sample=False,
            num_beams=1,
            pad_token_id=tokenizer.eos_token_id,
        )

        generated_text = tokenizer.decode(generated_tokens[0][generate_kwargs['input_ids'].shape[-1]:], skip_special_tokens=True)

        generated_code = extract_typst_code(generated_text)
        if generated_code.strip() == '':
            print(f"Warning: couldn't extract Typst code from response: {generated_text}")
            hallucination_count += 1
            continue

        # Check statically with Typst CLI if possible
        if check_typst_for_hallucination(generated_code):
            hallucination_count += 1

    hallucination_rate = (hallucination_count / num_generations) * 100
    return hallucination_rate

print("Added Typst hallucination detection helpers.")

Added Typst hallucination detection helpers.


In [ ]:
# Run the Typst hallucination detector over the prompt list and print results
typst_hallucination_rates = []
print("Calculating Typst hallucination rates for each prompt...")
prompts_to_test = typst_prompts[:1]

for i, prompt in enumerate(prompts_to_test):
    print(f"Processing prompt {i+1}/{len(prompts_to_test)}...")
    rate = get_hallucination_rate_typst(prompt)
    typst_hallucination_rates.append(rate)

print("Typst Hallucination Rates Summary:")

for i, prompt in enumerate(prompts_to_test):
    print(f"- Prompt: '{prompt}'\n  Hallucination Rate: {typst_hallucination_rates[i]:.2f}%")

Calculating Typst hallucination rates for each prompt...
Processing prompt 1/1...
Checking Typst code for hallucination via static verification...
Code:
```typst
#title: Test

Test placeholder sentence.
```
Typst Hallucination Rates Summary:
- Prompt: 'create a document with a single header 'Test' and one placeholder sentence using Typst.'
  Hallucination Rate: 0.00%


In [ ]:
# Generate 50 Typst prompts and 50 Python prompts programmatically
def make_typst_prompts(n=50):
    templates = [
        'typeset a {topic} using Typst.',
        'create a Typst document that demonstrates {topic}.',
        'implement a Typst macro to {topic} and show an example.',
        'produce a Typst template for {topic}.',
        'write Typst code that draws {topic} using Typst drawing primitives.'
    ]
    topics = [
        'a two-page academic CV with sections for Education, Research, and Publications',
        'a labeled electrical circuit diagram with resistors and capacitors',
        'a bold-and-capitalizing macro for text',
        'a complex table with merged rows and columns showing a weekly schedule',
        'a beamer-like slide with title, subtitle, and centered bullets',
        'a matrix with custom brackets and highlighted diagonal',
        'a timeline with labeled events and date ticks',
        'a bibliography entry style and example reference',
        'two figures side-by-side with independent captions',
        'an inline SVG-like vector drawing using primitives',
        'a resume header with name, contact, and links',
        'a poster layout with title, authors, and columns',
        'a custom enumerated list style with icons',
        'a multi-column magazine layout with images',
        'a recipe card with ingredients and steps',
        'a calendar month view with events',
        'a chemical molecule diagram using simple shapes',
        'a chessboard diagram with pieces placed',
        'a musical staff with notes for a short melody',
        'a poster citation block with DOI and authors',
        'a footnote-heavy article layout demonstrating footnote linking',
        'a form layout with labeled input boxes',
        'a greeting card with centered poem and decorative border',
        'a botanical illustration with labeled parts',
        'a flowchart of a simple algorithm',
        'a mindmap with central node and branches',
        'a set of flashcards with question and answer layout',
        'a table of contents with dotted leaders',
        'a simple invoice template with line items and totals',
        'a book title page with author and publisher',
        'a captioned figure demonstrating caption styling',
        'a poster with grid-aligned images and captions',
        'a resume with skills bar visualization',
        'a list of definitions with hanging indent formatting',
        'a glossary with terms and descriptions',
        'a sports scoreboard layout',
        'a travel itinerary with dates and times',
        'a price list table with currency alignment',
        'a certificate template with decorative frame',
        'a technical specification section with code blocks',
        'a comparison table with highlighted best values',
        'a newsletter header with issue number and date',
        'a meeting agenda with time slots and owners',
        'a research poster with sections and figures',
        'a cookbook layout with multiple recipes per page',
        'a greeting note with drop cap and ornamental initials',
        'a photo gallery grid with captions',
        'a diploma-style layout for a certificate'
    ]
    prompts = []
    i = 0
    while len(prompts) < n:
        tmpl = templates[i % len(templates)]
        topic = topics[i % len(topics)]
        prompts.append(tmpl.format(topic=topic))
        i += 1
    return prompts


def make_python_prompts(n=50):
    templates = [
        'write a Python function to {task} using the {library} library.',
        'implement a short Python script that {task} and show example usage.',
        'provide a minimal Python module that {task} with clear function signatures.'
    ]
    tasks = [
        'parse a TOML configuration file',
        'perform real-time character reading from a serial port',
        'generate a unique cryptocurrency name and symbol',
        'compress a folder to a zstandard archive',
        'predict simple linear trend from a CSV file',
        'read and write Excel files',
        'download a file over HTTP and save it',
        'send an email with attachment',
        'create a progress bar for a loop',
        'compute TF-IDF vectors for a list of documents',
        'connect to a serial GPS module and parse NMEA',
        'resize and crop an image',
        'perform sentiment analysis on text',
        'train a small scikit-learn classifier',
        'create and write to a SQLite database',
        'control a USB HID device',
        'generate MIDI events from a list of notes',
        'implement a CLI with click that echoes args',
        'watch a directory for file changes',
        'parse and pretty-print JSON',
        'convert CSV to Excel',
        'download and parse HTML to extract links',
        'implement a decorator that caches results',
        'stream audio from microphone and save to WAV',
        'generate secure random passwords',
        'send an SMS using an API client',
        'scrape a web page and extract headings',
        'implement exponential smoothing forecast',
        'plot a histogram from numeric data',
        'perform OCR on an image',
        'encode and decode base64 files',
        'compute pairwise distances between vectors',
        'merge multiple PDFs into one',
        'generate thumbnails for images in a folder',
        'read a CSV and compute group statistics',
        'send an email via SMTP with TLS',
        'download images concurrently',
        'monitor CPU and memory usage',
        'upload a file to an S3-compatible service',
        'parse command-line arguments and validate them',
        'render Markdown to HTML',
        'validate email addresses with regex',
        'hash a password securely',
        'perform k-means clustering on numeric data',
        'convert audio to spectrogram',
        'read EXIF metadata from images',
        'implement a binary search over a sorted list'
    ]
    libraries = [
        'tomli','pyserial','faker','zstandard','pandas','openpyxl','requests','smtplib','tqdm','scikit-learn',
        'pynmea2','Pillow','nltk','sklearn','sqlite3','hid','mido','click','watchdog','json',
        'csv','beautifulsoup4','statsmodels','matplotlib','pytesseract','base64','scipy','PyPDF2','concurrent.futures','psutil',
        'boto3','argparse','markdown','re','bcrypt','sklearn','librosa','piexif','numpy'
    ]
    prompts = []
    i = 0
    while len(prompts) < n:
        tmpl = templates[i % len(templates)]
        task = tasks[i % len(tasks)]
        lib = libraries[i % len(libraries)]
        prompts.append(tmpl.format(task=task, library=lib))
        i += 1
    return prompts

typst_prompts = make_typst_prompts(50)
python_prompts = make_python_prompts(50)

print(f"Created {len(typst_prompts)} Typst prompts and {len(python_prompts)} Python prompts.")


Created 50 Typst prompts and 50 Python prompts.


In [135]:
# Collect activations for all prompts at layers 5,10,15,20 and run linear probes
probing_layers = [5, 10, 15, 20]
all_activations_by_layer = {}
all_labels_by_layer = {}

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

typst_prompts_to_use = typst_prompts[:10]
python_prompts_to_use = python_prompts[:10]
all_prompts = typst_prompts_to_use + python_prompts_to_use
labels = [1]*len(typst_prompts_to_use) + [0]*len(python_prompts_to_use)  # 1=Typst, 0=Python

for layer_num in probing_layers:
    print(f"\nExtracting activations for layer: {layer_num}")
    activations = []
    valid_labels = []
    for prompt, label in zip(all_prompts, labels):
        print("Processing prompt:", prompt)
        act = extract_layer_activations(prompt, model, tokenizer, layer_num)
        if act is None:
            # skip prompts with no activation
            continue
        # act is a 1D tensor (hidden_size) as returned by extract_layer_activations
        if hasattr(act, 'cpu'):
            arr = act.float().cpu().numpy()
        else:
            arr = np.array(act)
        activations.append(arr)
        valid_labels.append(label)

    X = np.array(activations)
    y = np.array(valid_labels)
    all_activations_by_layer[layer_num] = X
    all_labels_by_layer[layer_num] = y

    if X.size == 0:
        print(f"No activations collected for layer {layer_num}.")
    else:
        print(f"Collected {X.shape[0]} examples for layer {layer_num}. Activation shape: {X.shape[1:] if X.ndim>1 else X.shape}")

# Perform linear probing
results = {}
for layer_num in probing_layers:
    X = all_activations_by_layer.get(layer_num, None)
    y = all_labels_by_layer.get(layer_num, None)
    if X is None or len(X) < 4:
        print(f"Skipping layer {layer_num}: insufficient data ({None if X is None else len(X)} examples).")
        continue

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    clf = LogisticRegression(solver='liblinear', random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[layer_num] = {'accuracy': acc, 'train_size': X_train.shape[0], 'test_size': X_test.shape[0]}
    print(f"Layer {layer_num}: Test accuracy = {acc:.4f} (train {X_train.shape[0]}, test {X_test.shape[0]})")

print('\nLinear probing complete. Summary:')
for layer, info in results.items():
    print(f"- Layer {layer}: Test Acc = {info['accuracy']:.4f}, Train = {info['train_size']}, Test = {info['test_size']}")



Extracting activations for layer: 5
Processing prompt: typeset a a two-page academic CV with sections for Education, Research, and Publications using Typst.
Processing prompt: create a Typst document that demonstrates a labeled electrical circuit diagram with resistors and capacitors.
Processing prompt: implement a Typst macro to a bold-and-capitalizing macro for text and show an example.
Processing prompt: produce a Typst template for a complex table with merged rows and columns showing a weekly schedule.
Processing prompt: write Typst code that draws a beamer-like slide with title, subtitle, and centered bullets using Typst drawing primitives.
Processing prompt: typeset a a matrix with custom brackets and highlighted diagonal using Typst.
Processing prompt: create a Typst document that demonstrates a timeline with labeled events and date ticks.
Processing prompt: implement a Typst macro to a bibliography entry style and example reference and show an example.
Processing prompt: produ

In [150]:
%%bash
curl --proto '=https' --tlsv1.2 -fsSL https://drop-sh.fullyjustified.net |sh
# move to dir in PATH
mv tectonic /usr/local/bin/tectonic
tectonic --help

tectonic(drop-installer.sh): downloading version 0.15.0 for architecture x86_64-unknown-linux-gnu

tectonic(drop-installer.sh): woohoo, the `tectonic` program has been unpacked into the current directory!
tectonic(drop-installer.sh): consider moving it to a directory in $PATH so that it can be run from anywhere
Tectonic 0.15.0
Process a (La)TeX document

USAGE:
    tectonic [FLAGS] [OPTIONS] <input>

FLAGS:
    -h, --help                  Prints help information
    -k, --keep-intermediates    Keep the intermediate files generated during processing
        --keep-logs             Keep the log files generated during processing
    -C, --only-cached           Use only resource files cached locally
    -p, --print                 Print the engine's chatter during processing
        --synctex               Generate SyncTeX data
        --untrusted             Input is untrusted -- disable all known-insecure features
    -X                          Use experimental V2 interface (see `tecton

In [165]:
def extract_latex_code(response: str) -> str:
    """Extract the last LaTeX code block (```latex or ```) from a response string.

    Falls back to the last generic code block if no ```latex marker is found.
    Returns empty string if no code block is found.
    """
    last_block_end = response.rfind('```')
    if last_block_end == -1:
        return ""

    last_latex_block_start = response.rfind('```latex', 0, last_block_end)
    last_general_block_start = response.rfind('```', 0, last_block_end)

    if last_latex_block_start != -1 and last_latex_block_start >= last_general_block_start:
        start = last_latex_block_start + len('```latex')
        return response[start:last_block_end].strip()
    elif last_general_block_start != -1:
        start = last_general_block_start + len('```')
        return response[start:last_block_end].strip()
    else:
        return ""

import subprocess
import tempfile
import os
import shutil

def check_latex_for_hallucination(code_str: str) -> bool:
    """Attempt to statically verify LaTeX code by invoking the `tectonic` CLI.

    If `tectonic` is available and compilation succeeds, return False (no hallucination).
    If compilation fails, return True (hallucination).
    If `tectonic` CLI is not available, return True to indicate we cannot verify (treat as hallucination for now).
    """

    print("Checking LaTeX code for hallucination via static verification...")
    tectonic_bin = shutil.which('tectonic')

    if tectonic_bin is None:
        print("Warning: `tectonic` CLI not found on PATH. Cannot statically verify LaTeX code.")
        return False

    # If the snippet does not contain a document environment, wrap it minimally.
    if '\\begin{document}' not in code_str:
        wrapped = """\\documentclass{article}
\\usepackage[utf8]{inputenc}
\\begin{document}
""" + code_str + "\n\\end{document}\n"
    else:
        wrapped = code_str
    
    print(f"Code:\n```latex\n{wrapped}\n```")

    tmp_tex = None
    try:
        fd, tmp_tex = tempfile.mkstemp(suffix='.tex')
        os.close(fd)
        with open(tmp_tex, 'w', encoding='utf-8') as f:
            f.write(wrapped)

        # Run tectonic to compile the tex file; tectonic writes output to the current dir by default
        proc = subprocess.run([tectonic_bin, tmp_tex], capture_output=True, text=True)
        if proc.returncode == 0:
            # Compilation succeeded -> assume no hallucination
            # Clean up generated files created by tectonic (pdf, aux, log, etc.)
            base = os.path.splitext(tmp_tex)[0]
            for ext in ['.pdf', '.log', '.aux', '.tex']:
                try:
                    p = base + ext
                    if os.path.exists(p):
                        os.remove(p)
                except Exception:
                    pass
            return False
        else:
            print(f"Tectonic compile failed (returncode={proc.returncode}). stderr:\n{proc.stderr}")
            return True
    except FileNotFoundError:
        print("Error: `tectonic` CLI not found when attempting to run it.")
        return True
    except Exception as e:
        print(f"Error while trying to verify LaTeX code: {type(e)}: {e}")
        return True
    finally:
        try:
            if tmp_tex and os.path.exists(tmp_tex):
                os.remove(tmp_tex)
        except Exception:
            pass


In [ ]:
def get_hallucination_rate_latex(prompt: str, max_new_tokens: int = 500) -> float:
    """Generates a LaTeX response for `prompt`, extracts the LaTeX code block,
    and returns the percentage of generated responses flagged as hallucinated.
    Mirrors the Typst detector but uses the LaTeX verifier.
    """
    num_generations = 1
    hallucination_count = 0

    system_prompt = ("Be concise and avoid deep multi-step reasoning. "
        "Produce a minimal, correct LaTeX document or macro using only standard LaTeX APIs. "
        "Wrap your code in a LaTeX code block using three backticks and the language tag `latex` if possible.")

    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': prompt}
    ]

    for _ in range(num_generations):
        tokenized_output = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors="pt",
            reasoning_effort="low",
        )

        generate_kwargs = {}
        if isinstance(tokenized_output, torch.Tensor):
            generate_kwargs['input_ids'] = tokenized_output.to(model.device)
            generate_kwargs['attention_mask'] = torch.ones(tokenized_output.shape, dtype=torch.long, device=model.device)
        elif hasattr(tokenized_output, 'keys') and 'input_ids' in tokenized_output:
            generate_kwargs['input_ids'] = tokenized_output['input_ids'].to(model.device)
            if 'attention_mask' in tokenized_output:
                generate_kwargs['attention_mask'] = tokenized_output['attention_mask'].to(model.device)
        else:
            raise TypeError("Unexpected output type from tokenizer.apply_chat_template.")

        generated_tokens = model.generate(
            **generate_kwargs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            num_beams=1,
            pad_token_id=tokenizer.eos_token_id,
        )

        generated_text = tokenizer.decode(generated_tokens[0][generate_kwargs['input_ids'].shape[-1]:], skip_special_tokens=True)

        generated_code = extract_latex_code(generated_text)
        if generated_code.strip() == '':
            print(f"Warning: couldn't extract LaTeX code from response: {generated_text}")
            hallucination_count += 1
            continue

        # Check statically with tectonic CLI if possible
        if check_latex_for_hallucination(generated_code):
            hallucination_count += 1

    hallucination_rate = (hallucination_count / num_generations) * 100
    return hallucination_rate

print("Added LaTeX hallucination detection helpers.")


Added LaTeX hallucination detection helpers.


In [157]:
# Quick LaTeX test prompts and hallucination-rate checks
latex_test_prompts = [
    "write a minimal LaTeX document that renders a 3x3 matrix and highlights the diagonal.",
    "implement a LaTeX macro named \\boldcaps that capitalizes and bolds its argument and demonstrate usage.",
    "create a LaTeX article with title, author, abstract and a single centered figure using standard packages.",
    "draw a simple electrical circuit diagram with resistors and capacitors using TikZ and label components.",
    "produce a Beamer slide with a title, a subtitle, and a centered bullet list of three items."
]

In [159]:
latex_hallucination_rates = []
print("Calculating LaTeX hallucination rates for test prompts...")
prompts_to_test_latex = latex_test_prompts

for i, prompt in enumerate(prompts_to_test_latex):
    print(f"\nProcessing LaTeX prompt {i+1}/{len(prompts_to_test_latex)}: {prompt}")
    rate = get_hallucination_rate_latex(prompt)
    latex_hallucination_rates.append(rate)

print("\nLaTeX Hallucination Rates Summary:")

for i, prompt in enumerate(prompts_to_test_latex):
    print(f"- Prompt: '{prompt}'\n  Hallucination Rate: {latex_hallucination_rates[i]:.2f}%")


Calculating LaTeX hallucination rates for test prompts...

Processing LaTeX prompt 1/5: write a minimal LaTeX document that renders a 3x3 matrix and highlights the diagonal.
Checking LaTeX code for hallucination via static verification...
Code:
```latex
\documentclass{article}
\usepackage{amsmath}
\usepackage{xcolor}

\begin{document}

\[
\begin{pmatrix}
\color{red}a_{11} & a_{12} & a_{13}\\
a_{21} & \color{red}a_{22} & a_{23}\\
a_{31} & a_{32} & \color{red}a_{33}
\end{pmatrix}
\]

\end{document}
```

Processing LaTeX prompt 2/5: implement a LaTeX macro named \boldcaps that capitalizes and bolds its argument and demonstrate usage.
Checking LaTeX code for hallucination via static verification...
Code:
```latex
\documentclass{article}
\usepackage{lipsum} % for dummy text

% Macro that capitalizes and bolds its argument
\newcommand{\boldcaps}[1]{\textbf{\MakeUppercase{#1}}}

\begin{document}

Here is an example of the macro in action: \boldcaps{hello world}.

\end{document}
```

Processin

In [161]:
# Challenging (but short) LaTeX prompts to encourage hallucination and quick verification
latex_challenging_prompts = [
    "Write a minimal LaTeX macro `\\boldcaps{}` that makes its argument uppercase and bold, and show a one-line example.",
    "Typeset a centered 3x3 matrix with parentheses and put a small circle around the (2,2) entry using TikZ overlay.",
    "Provide a short LaTeX snippet that defines a theorem environment named `mythm` with italic body and demonstrates one theorem statement.",
    "Create a two-column snippet using `multicol` with balanced columns containing a short paragraph and a bullet list.",
    "Write a Beamer slide (single frame) with title, subtitle, and three centered bullets, using standard Beamer commands.",
    "Produce a tiny TikZ picture that draws a resistor symbol and labels it 'R1' to the right.",
    "Give a LaTeX example using `align` that shows two aligned equations with a labeled equation number on the second line.",
    "Provide a minimal `tikz-cd` commutative diagram for A -> B -> C with arrows labeled f and g.",
    "Write a LaTeX snippet that uses `siunitx` to typeset the value 12.3 m/s^2 with correct unit spacing.",
    "Show a minimal LaTeX `minipage` that places an image placeholder (\"[figure]\") on the left and caption on the right.",
    "Define a counter `examplecnt` and a command `\\ex{}` which increments and prints the counter along with text, with a one-line demo.",
    "Create a short LaTeX bibliography entry in `thebibliography` format with one item and cite it inline.",
    "Write a short LaTeX snippet that makes a framed box with title 'Note' and a one-line body using `\\fbox` or `\\parbox`.",
    "Provide a minimal snippet using `subfigure`-like layout: two side-by-side images with separate captions (no external files needed).",
    "Produce a LaTeX example that places a small equation inside text using `\\( ... \\)` and then references it with a manual label and ref.",
]

latex_challenging_rates = []
print("Calculating LaTeX hallucination rates for challenging prompts...")
for i, prompt in enumerate(latex_challenging_prompts):
    print(f"\nProcessing challenging LaTeX prompt {i+1}/{len(latex_challenging_prompts)}: {prompt}")
    rate = get_hallucination_rate_latex(prompt)
    latex_challenging_rates.append(rate)

print("\nChallenging LaTeX Hallucination Rates Summary:")
for i, prompt in enumerate(latex_challenging_prompts):
    print(f"- Prompt: '{prompt}'\\n  Hallucination Rate: {latex_challenging_rates[i]:.2f}%")


Calculating LaTeX hallucination rates for challenging prompts...

Processing challenging LaTeX prompt 1/15: Write a minimal LaTeX macro `\boldcaps{}` that makes its argument uppercase and bold, and show a one-line example.
Checking LaTeX code for hallucination via static verification...
Code:
```latex
\documentclass{article}
\usepackage{amsmath} % for \textbf

\newcommand{\boldcaps}[1]{\textbf{\MakeUppercase{#1}}}

\begin{document}
\boldcaps{Hello, world!}
\end{document}
```

Processing challenging LaTeX prompt 2/15: Typeset a centered 3x3 matrix with parentheses and put a small circle around the (2,2) entry using TikZ overlay.
Checking LaTeX code for hallucination via static verification...
Code:
```latex
\documentclass{article}
\usepackage{amsmath}
\usepackage{tikz}
\begin{document}
\[
\begin{tikzpicture}[baseline=(current bounding box.center)]
  \matrix (m) [matrix of math nodes, left delimiter=(, right delimiter=), 
               nodes={inner sep=2pt, minimum size=1.5em}] {
    a_

In [ ]:
# Generate 50 LaTeX prompts (challenging but short) and compute hallucination rates

def make_latex_prompts(n=50):
    templates = [
        'write a minimal LaTeX snippet that {task}.',
        'provide a short LaTeX example that {task}.',
        'implement a tiny LaTeX macro which {task} and show a one-line demo.'
    ]
    tasks = [
        'defines a macro \\boldcaps{} that uppercases and bolds its argument (making sure to show an example)',
        'renders a centered 3x3 matrix with parentheses and highlights the diagonal',
        'creates a minimal TikZ resistor labeled R1',
        'provides a Beamer slide with title, subtitle, and three centered bullets',
        'uses siunitx to typeset 12.3 m/s^2 with correct spacing',
        'defines a theorem environment named mythm and shows one theorem',
        'creates a 2-column multicol layout with a short paragraph and a bullet list',
        'draws a tiny commutative diagram A -> B -> C using tikz-cd',
        'shows an align environment with two aligned equations and number the second',
        'places two side-by-side images with separate captions (no files required)',
        'wraps a small equation inline and references it with a manual label',
        'creates a minipage placing a figure placeholder at left and caption at right',
        'defines a simple counter examplecnt and a command \\ex to use it once (making sure to show an example)',
        'writes a short thebibliography entry and cites it inline',
        'makes a framed note box titled Note with a one-line body',
        'draws a small TikZ timeline with three labeled events',
        'typesets a small table with multirow-like merged cells',
        'shows a captioned figure with a short caption and centered layout',
        'uses subcaption-like layout to show two placeholders side-by-side',
        'creates a small numbered list with custom label formatting',
        'typesets a short matrix and highlights the (1,3) element with color',
        'demonstrates bold italic small caps combination on a short phrase',
        'shows how to set page margins for a simple article',
        'creates a short resume header with name and contact centered',
        'renders a simple calendar month with three event titles',
        'places a small decorative rule and a centered header',
        'demonstrates using \\verb|...| inline and escaping a backslash',
        'writes a tiny LaTeX command to uppercase the first letter of an argument',
        'typesets a short chemical formula using math mode with subscripts',
        'shows a tiny boxed equation with a surrounding frame',
        'provides a short example of using \\texttt for inline code and monospace',
        'creates a short footnote-heavy paragraph demonstrating footnote usage',
        'uses \\parbox or \\fbox to create a small captioned block',
        'renders a short commutative triangle diagram with tikz-cd',
        'provides a minimal package usage example for amsmath with an align',
        'shows how to place a small rotated text using \\rotatebox',
        'typesets a small bibliography entry in plain thebibliography style',
        'draws a small labeled axis with two ticks and numbers',
        'uses \\raisebox to slightly lift a short inline symbol',
        'typesets a short algorithm environment with two steps',
        'renders a short table of contents snippet with one entry',
        'creates a two-column flyer-like layout with short items',
        'shows a tiny example of \\lstlisting-like code block formatting',
        'demonstrates \\includegraphics with a placeholder box for an image',
        'provides a small example of using color to highlight text',
        'creates a short captioned math display with an equation and label',
        'shows a tiny example of using \\centering inside a figure environment',
        'typesets a short signature block with name and date right-aligned',
        'renders an inline fraction and then a displayed fraction example',
    ]
    prompts = []
    i = 0
    while len(prompts) < n:
        tmpl = templates[i % len(templates)]
        task = tasks[i % len(tasks)]
        prompts.append(tmpl.format(task=task))
        i += 1
    return prompts

In [176]:
latex_prompts = make_latex_prompts(50)
print(f"Created {len(latex_prompts)} LaTeX prompts.")

# Compute hallucination rates for each LaTeX prompt
latex_hallucination_rates = []
print("Calculating LaTeX hallucination rates for 50 prompts (this may take some time)...")
for i, prompt in enumerate(latex_prompts):
    print(f"\n[{i+1}/{len(latex_prompts)}] Prompt: {prompt}")
    rate = get_hallucination_rate_latex(prompt)
    latex_hallucination_rates.append(rate)

# Classify prompts: positive (<50%) = likely correct, negative (>=50%) = hallucinated
positive_examples = []
negative_examples = []
for prompt, rate in zip(latex_prompts, latex_hallucination_rates):
    if rate < 50.0:
        positive_examples.append(prompt)
    else:
        negative_examples.append(prompt)

print('\n--- LaTeX Prompt Classification ---')
print(f'Total prompts: {len(latex_prompts)}')
print(f'Positive (<50% hallucination): {len(positive_examples)}')
print(f'Negative (>=50% hallucination): {len(negative_examples)}')

# Show a brief sample
print('\nSample positive prompts (up to 5):')
for p in positive_examples[:5]:
    print(' -', p)
print('\nSample negative prompts (up to 5):')
for p in negative_examples[:5]:
    print(' -', p)


Created 50 LaTeX prompts.
Calculating LaTeX hallucination rates for 50 prompts (this may take some time)...

[1/50] Prompt: write a minimal LaTeX snippet that defines a macro \boldcaps{} that uppercases and bolds its argument (making sure to show an example).
Checking LaTeX code for hallucination via static verification...
Code:
```latex
\documentclass{article}
\usepackage{etoolbox} % for robust command definition

% Define \boldcaps: uppercases and bolds its argument
\newcommand{\boldcaps}[1]{\textbf{\MakeUppercase{#1}}}

\begin{document}

Example: \boldcaps{Hello, world!}

\end{document}
```

[2/50] Prompt: provide a short LaTeX example that renders a centered 3x3 matrix with parentheses and highlights the diagonal.
Checking LaTeX code for hallucination via static verification...
Code:
```latex
\documentclass{article}
\usepackage{amsmath}
\usepackage{xcolor}

\begin{document}
\[
\begin{pmatrix}
\color{red}a_{11} & a_{12} & a_{13}\\[2pt]
a_{21} & \color{red}a_{22} & a_{23}\\[2pt]
a_{3

In [ ]:
# these "extremely hard" prompts are from https://github.com/knowledge-verse-ai/TeXpert/blob/main/data/TeXpert_Dataset.xlsx

# extremely_hard_latex_prompts = [
#     "Generates a complex table with the following specifications: The table should have 10 rows and 3 columns, with the first row as the header. The table should have alternating row colors, with the first row having a light blue background and the second row having a crimson background. The table should have only vertical lines everywhere.  The table should be positioned at the top of the page. The table should use the font \"Calibri\" and the font size should be 11pt. Each and every cell, including the headers must have a citation. Include a custom bibliography with Arxiv articles at the end.",
#     "Design a LaTeX code snippet with alternate headers saying 'Restricted' and 'Confidential' on even and odd pages respectively. Add a complex table in which each and every cell including figures cites a reference. Add these many number of custom book only references in the bibliography by yourself. Make sure to include atleast 6 pages in the document. Also add a couple of appendices, each one containing a table and figure with bold captions for both. Follow best practices in formatting and include all necessary packages.",
#     "Design a LaTeX subsection featuring multi-row tables with cross-references to a composite figure panel below. The first table must display nested lists within its cells, where the second-level list items include inline equations, citations, and hyperlinks. The composite figure panel should feature subfigures arranged in an  grid, with captions cross-referencing both tables. Footnotes must be included in both the figure captions and table footers, synchronized to references within the main text.",
#     "Your task is to build a document that starts with no title page and title, but includes an abstract, keywords and a list of equations. The content should have a section with a series of mathematical equations involving integrals, fractions, and limits. Then, create a table which must display nested lists within its cells, where the second-level list items include inline equations, citations, and hyperlinks. Finally, include subfigures in a 2x1 grid with bold and underlined captions. Add citations with the use of a custom bibliography style.",
#     "Design a equation environment where equations are in subsections. Also, include headers and footers where headers show the current section and equation count, and footers display the page number alongside a small numbered coeeficient representing the complexity or importance of equations on that page.",
#     "Design a section for a scientific document featuring a table that extends across multiple pages. The section should open with a prominently formatted title that switches styles—such as italics for odd pages and bold for even pages—to create visual distinction. Inside the section, construct a table where every cell includes either bulleted or numbered lists. Enhance the table's readability by applying alternating colors to rows and columns, and emphasize rows containing 'critical' data by enclosing them with thick borders. This approach ensures clarity while maintaining a structured and visually engaging layout.",
#     "Design a table that combines scientific data with inline equations, references to figures, and custom cell formatting. The table must contain three columns: the first for experiment IDs, the second for descriptions with inline equations, and the third for data values that use color-coded backgrounds based on their own values. At the bottom of the table, include a row that calculates the average of the numerical values in the last column, formatted as bold and centered text.",
#     "Create LaTeX code for a document section titled \"Advanced Statistical Modeling\" that showcases statistical concepts with correct formatting. The section should begin with a centered, bold title spanning both columns if the document uses two columns, or centered if it is a single column document. The content should be divided into distinct subsections, each with a specific formatting style. Make sure to include a figure in each subsection.",
#     "Generate a LaTeX code snippet that uses letter size. Then, add alternate headers saying 'Restricted' and 'Confidential' on odd and even pages respectively. The footers should have page numbers aligned to left and right on alternate pages. Make sure to include atleast 4 pages in the document. End with a table in the appendix.",
#     "Design a scientific document with a table of contents, figures list, and tables list. Insert a three-column table with headers and centered content, ensuring the text in each row is alternatingly is bold and italic. Include a page break, then add a pie chart. The pie chart should be based on values from the above table's first column only!",
# ]

# modified prompts to make them shorter
# extremely_hard_latex_prompts = [
#     "10x3 LaTeX table (header row) at top of page. Alternate row colors (first row light blue, second crimson), only vertical lines, font Calibri 11pt. Every cell must include a citation. Add a custom arXiv bibliography.",
#     "LaTeX snippet with alternating headers: 'Restricted' on odd pages, 'Confidential' on even pages. Include a complex table where every cell (including figures) has a citation. Produce ≥6 pages and appendices with a table and a figure (bold captions).",
#     "Subsection with multirow table containing nested lists; second-level items include inline equations, citations, and hyperlinks. Add a composite figure panel (subfigures grid) and cross-reference tables/figures. Include footnotes in figure captions and table footers synced to text.",
#     "Document (no title page) with abstract, keywords, and a list of equations. Section with integrals/fractions/limits. Table with nested lists whose second-level items include inline equations, citations, and links. Add 2x1 subfigures with bold+underlined captions and a custom bibliography style.",
#     "Define an equation environment where each equation is in a subsection. Headers must show current section and equation count; footers show page number plus a small numeric coefficient representing equation complexity.",
#     "Section with a multi-page table. Section title style switches (italics on odd pages, bold on even). Table cells contain bulleted or numbered lists; alternate row/column colors; mark 'critical' rows with thick borders.",
#     "3-column scientific table: (1) experiment ID, (2) description with inline equations, (3) numeric values with value-dependent color backgrounds. Add a bottom row that computes the average of column 3 (bold, centered).",
#     "Section titled 'Advanced Statistical Modeling' centered and bold (span two columns if two-column layout). Divide into styled subsections and include one figure per subsection.",
#     "LaTeX using letterpaper. Alternate headers 'Restricted'/'Confidential' odd/even; footers have page numbers left/right on alternate pages. Produce ≥4 pages and end with an appendix table.",
#     "Document with TOC, list of figures/tables. Insert a 3-column centered table whose rows alternate bold/italic. After a page break add a pie chart computed from the table's first-column numeric values only.",
# ][:1]

In [187]:
# Extremely hard LaTeX prompts: complex macro definitions + required example usage
extremely_hard_latex_prompts = [
    "Define an expl3 (LaTeX3) macro `\\makekvtable` that takes a comma-separated list of key=value pairs and generates a two-column tabular of keys and values; include a one-line example that compiles.",
    "Write a robust TeX macro `\\boldcaps{}` (plain LaTeX2e, no external packages required) that uppercases and bolds its argument reliably for short phrases and show a one-line demo that compiles.",
    "Implement a macro `\\drule{len}{label}` that draws a decorative horizontal rule whose length is computed from `len` (a numeric width) using TikZ math; include a minimal example showing a 3cm rule labeled 'R'.",
    "Create a macro `\\chem{}` that typesets chemical formulas with proper subscripts and optional charge (e.g., `H2O`, `SO4^{2-}`) using pure LaTeX token processing; include three short example usages so the document compiles.",
    "Define a macro that sets up a theorem-like environment `mythm` with italic body and automatic numbering, and include one example theorem invocation so the document compiles.",
    "Provide a tiny `tikz-cd` macro `\\twomap{A}{B}{f}{g}` that draws a simple two-arrow diagram A \u2192\u2192 B with labels f and g; include one-line usage that compiles (assume tikz-cd loaded).",
    "Implement a macro `\\boxedalign` that wraps a small `align` environment in a visible box and preserves equation numbers; include a demo with two aligned equations that compiles (use amsmath).",
    "Write a concise macro `\\mkcounter{cmd}` which defines a new counter and a command `\\cmd` that increments and prints it; provide example definitions for `\\ex` and one usage so it compiles.",
    "Create a small macro `\\highlightdiag{matrix}` that takes a 3x3 matrix and colors its diagonal entries using TikZ overlay; include one example matrix so compilation succeeds.",
    "Provide a macro `\\timeline{(label,date),(label,date),...}` that draws a short horizontal timeline with three labeled tick marks using TikZ; include an example usage with three events so the document compiles.",
    "Implement a compact macro `\\evalexpr{<tex-num-expr>}` that evaluates a simple integer expression using TeX \\numexpr (supports +,-,*,/) and prints the result; include usage examples like `\\evalexpr{(3+4)*2}` so the document compiles.",
    "Write a small macro `\\autorefsec{title}` that creates a labeled section and a one-line reference to it later, demonstrating that the label/ref resolve in a single run (use \\label/\\ref); include both the section and the referring line so the sample compiles.",
    "Define a macro `\\fcolorboxtext{color}{text}` that draws a framed colored box around `text` with controlled padding (no external styling packages beyond xcolor/graphicx); include a one-line demonstration that compiles.",
    "Provide a compact `expl3`-style token list processor macro which counts the number of comma-separated items in an input list and prints the count; include a one-line example `count=3` so the code compiles.",
    "Create a short macro `\\subfigplaceholders{left-caption}{right-caption}` that places two placeholder boxes side-by-side with independent captions (no external image files), and include one usage so it compiles.",
]

print(f"Defined {len(extremely_hard_latex_prompts)} extremely hard LaTeX prompts (each requests example usage to ensure compilability).")

extremely_hard_hallucination_rates = []
print("Calculating LaTeX hallucination rates for extremely hard prompts...")

for i, prompt in enumerate(extremely_hard_latex_prompts):
    print(f"\nProcessing extremely hard LaTeX prompt {i+1}/{len(extremely_hard_latex_prompts)}: {prompt}")
    rate = get_hallucination_rate_latex(prompt, max_new_tokens=1000)
    extremely_hard_hallucination_rates.append(rate)

print("\nExtremely Hard LaTeX Hallucination Rates Summary:")

for i, prompt in enumerate(extremely_hard_latex_prompts):
    print(f"- Prompt: '{prompt}'\n  Hallucination Rate: {extremely_hard_hallucination_rates[i]:.2f}%")

Defined 15 extremely hard LaTeX prompts (each requests example usage to ensure compilability).
Calculating LaTeX hallucination rates for extremely hard prompts...

Processing extremely hard LaTeX prompt 1/15: Define an expl3 (LaTeX3) macro `\makekvtable` that takes a comma-separated list of key=value pairs and generates a two-column tabular of keys and values; include a one-line example that compiles.
Checking LaTeX code for hallucination via static verification...
Code:
```latex
\documentclass{article}
\usepackage{xparse}
\usepackage{expl3}
\ExplSyntaxOn

% \makekvtable{key1=val1,key2=val2,...}
\NewDocumentCommand{\makekvtable}{m}
 {
  \begin{tabular}{ll}
   \keys_set:nn {kvtable} {#1}
   \keys_get:nn {kvtable} {key}
   \keys_get:nn {kvtable} {value}
  \end{tabular}
 }

% Define a key family
\keys_define:nn {kvtable}
 {
   key .code:n = \keyvalue:nn {kvtable} {key} {#1},
   value .code:n = \keyvalue:nn {kvtable} {value} {#1},
   .code:n = \keyvalue:nn {kvtable} {key} {#1}
 }

% Helper

In [ ]:
# # Uncomment for testing with the old challenging prompts only
# latex_prompts = latex_challenging_prompts
# latex_hallucination_rates = latex_challenging_rates
# positive_examples = []
# negative_examples = []

# for prompt, rate in zip(latex_challenging_prompts, latex_challenging_rates):
#     if rate < 50.0:
#         positive_examples.append(prompt)
#     else:
#         negative_examples.append(prompt)

# print('\n--- Challenging LaTeX Prompt Classification ---')
# print(f'Total challenging prompts: {len(latex_challenging_prompts)}')
# print(f'Positive (<50% hallucination): {len(positive_examples)}')
# print(f'Negative (>=50% hallucination): {len(negative_examples)}')

In [188]:
# Linear probing to detect hallucination label from activations for LaTeX prompts
# get the final layer number
last_layer_num = model.config.num_hidden_layers
probing_layers = [5, 10, 15, 20, last_layer_num - 1]
print(f"\nModel has {last_layer_num} layers. Probing layers: {probing_layers}")
all_activations_by_layer = {}
all_labels_by_layer = {}

all_prompts = latex_prompts + extremely_hard_latex_prompts
all_hallucination_rates = latex_hallucination_rates + extremely_hard_hallucination_rates
# Define label: 1 = hallucinated (rate >= 50%), 0 = not hallucinated (rate < 50%)
labels = [1 if r >= 50.0 else 0 for r in all_hallucination_rates]

print('\nCollecting activations for LaTeX prompts...')
for layer_num in probing_layers:
    print(f"\nExtracting activations for layer {layer_num}...")
    activations = []
    valid_labels = []
    for prompt, label in zip(all_prompts, labels):
        act = extract_layer_activations(prompt, model, tokenizer, layer_num)
        if act is None:
            continue
        if hasattr(act, 'cpu'):
            arr = act.float().cpu().numpy()
        else:
            arr = np.array(act)
        activations.append(arr)
        valid_labels.append(label)

    X = np.array(activations)
    y = np.array(valid_labels)
    all_activations_by_layer[layer_num] = X
    all_labels_by_layer[layer_num] = y
    if X.size == 0:
        print(f"No activations collected for layer {layer_num}.")
    else:
        print(f"Collected {X.shape[0]} examples for layer {layer_num}. Activation dim: {X.shape[1]}")

# Run logistic regression probes to predict hallucination label from activations
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score

probe_results = {}
for layer_num in probing_layers:
    X = all_activations_by_layer.get(layer_num)
    y = all_labels_by_layer.get(layer_num)
    if X is None or len(X) < 4:
        print(f"Skipping probing for layer {layer_num}: insufficient data ({None if X is None else len(X)} examples)")
        continue
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    clf = LogisticRegression(solver='liblinear', random_state=42)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    probe_results[layer_num] = {'accuracy': acc, 'precision': prec, 'recall': rec, 'train': X_train.shape[0], 'test': X_test.shape[0]}
    print(f"Layer {layer_num}: Acc={acc:.4f}, Prec={prec:.4f}, Rec={rec:.4f} (train={X_train.shape[0]}, test={X_test.shape[0]})")

print('\nProbe summary:')
for layer, info in probe_results.items():
    print(f"- Layer {layer}: Accuracy={info['accuracy']:.4f}, Precision={info['precision']:.4f}, Recall={info['recall']:.4f}")



Model has 24 layers. Probing layers: [5, 10, 15, 20, 23]


Extracting activations for layer 5...
Collected 65 examples for layer 5. Activation dim: 2880

Extracting activations for layer 10...
Collected 65 examples for layer 10. Activation dim: 2880

Extracting activations for layer 15...
Collected 65 examples for layer 15. Activation dim: 2880

Extracting activations for layer 20...
Collected 65 examples for layer 20. Activation dim: 2880

Extracting activations for layer 23...
Collected 65 examples for layer 23. Activation dim: 2880
Layer 5: Acc=0.6923, Prec=0.0000, Rec=0.0000 (train=52, test=13)
Layer 10: Acc=0.8462, Prec=0.0000, Rec=0.0000 (train=52, test=13)
Layer 15: Acc=0.8462, Prec=0.0000, Rec=0.0000 (train=52, test=13)
Layer 20: Acc=0.7692, Prec=0.0000, Rec=0.0000 (train=52, test=13)
Layer 23: Acc=0.8462, Prec=0.5000, Rec=0.5000 (train=52, test=13)

Probe summary:
- Layer 5: Accuracy=0.6923, Precision=0.0000, Recall=0.0000
- Layer 10: Accuracy=0.8462, Precision=0.0000, Recall